In [71]:
from langchain.tools import tool
from langchain_openai import ChatOpenAI 
from langchain_core.messages import HumanMessage
import requests
import os
from dotenv import load_dotenv

from typing import Annotated
from langchain_core.tools import InjectedToolArg

In [31]:
load_dotenv()

True

In [32]:
EXCHANGE_RATE_API_KEY = os.environ['EXCHANGE_RATE_API']

In [33]:
# Where USD is the base currency you want to use
currency = 'USD'
url = f'https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/latest/{currency}'
print(url)

https://v6.exchangerate-api.com/v6/9d114de34374e19986598ef8/latest/USD


In [34]:
response = requests.get(url)
data = response.json()
data

{'result': 'success',
 'documentation': 'https://www.exchangerate-api.com/docs',
 'terms_of_use': 'https://www.exchangerate-api.com/terms',
 'time_last_update_unix': 1754697601,
 'time_last_update_utc': 'Sat, 09 Aug 2025 00:00:01 +0000',
 'time_next_update_unix': 1754784001,
 'time_next_update_utc': 'Sun, 10 Aug 2025 00:00:01 +0000',
 'base_code': 'USD',
 'conversion_rates': {'USD': 1,
  'AED': 3.6725,
  'AFN': 68.6664,
  'ALL': 83.5745,
  'AMD': 383.7376,
  'ANG': 1.79,
  'AOA': 921.0297,
  'ARS': 1329.42,
  'AUD': 1.5329,
  'AWG': 1.79,
  'AZN': 1.7006,
  'BAM': 1.6794,
  'BBD': 2.0,
  'BDT': 121.5519,
  'BGN': 1.6794,
  'BHD': 0.376,
  'BIF': 2965.7471,
  'BMD': 1.0,
  'BND': 1.285,
  'BOB': 6.9136,
  'BRL': 5.4227,
  'BSD': 1.0,
  'BTN': 87.612,
  'BWP': 13.8935,
  'BYN': 3.089,
  'BZD': 2.0,
  'CAD': 1.3752,
  'CDF': 2888.4995,
  'CHF': 0.8078,
  'CLP': 972.8682,
  'CNY': 7.1876,
  'COP': 4072.0155,
  'CRC': 506.4492,
  'CUP': 24.0,
  'CVE': 94.6814,
  'CZK': 20.9814,
  'DJF': 177

In [35]:
data['conversion_rates']['INR']

87.6122

In [72]:
@tool
def get_conversion_factor(base_currency: str, target_currency:str) -> float:
    '''fetches the currency conversion factor between a given base currency and target currency'''
    url = f'https://v6.exchangerate-api.com/v6/{EXCHANGE_RATE_API_KEY}/latest/{base_currency}'
    response = requests.get(url)
    data = response.json()
    return data['conversion_rates'][target_currency]

@tool
def convert(base_currency_value: float, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    '''multiply base currency value with conversion rate to get target currency value'''
    return base_currency_value * conversion_rate

In [37]:
conversion_factor = get_conversion_factor.invoke({'base_currency':'USD','target_currency':'INR'})

In [38]:
answer = convert.invoke({'base_currency_value':12, 'conversion_rate':conversion_factor})

In [39]:
answer

1051.3464

### Tool binding

In [84]:
llm = ChatOpenAI(model='gpt-4o-mini', temperature=0.2)

In [85]:
llm_with_tool = llm.bind_tools([get_conversion_factor, convert])

In [86]:
query = HumanMessage('what is the conversion factor between USD and INR and based on that convert 900 dollars to indian rupees')
messages = [query]

In [87]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR and based on that convert 900 dollars to indian rupees', additional_kwargs={}, response_metadata={})]

In [88]:
ai_message = llm_with_tool.invoke(messages)
messages.append(ai_message)

In [89]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR and based on that convert 900 dollars to indian rupees', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_b2Fweh8ccEa3seesmngBAF7e', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_mo8Qazijq8VbpOraSLhMKci7', 'function': {'arguments': '{"base_currency_value": 900}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 108, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C2t85jXyiq2GrtDZgAqjr9s3dAvi3',

In [90]:
ai_message.tool_calls

[{'name': 'get_conversion_factor',
  'args': {'base_currency': 'USD', 'target_currency': 'INR'},
  'id': 'call_b2Fweh8ccEa3seesmngBAF7e',
  'type': 'tool_call'},
 {'name': 'convert',
  'args': {'base_currency_value': 900},
  'id': 'call_mo8Qazijq8VbpOraSLhMKci7',
  'type': 'tool_call'}]

In [91]:
for tool_call in ai_message.tool_calls:
    # print(tool_call)

    # execute the 1st tool and get the value of conversion rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # print(tool_message1)
        # fetch the conversion rate
        # print(tool_message1.content)
        conversion_rate = tool_message1.content
        # append the tool message to messages 
        messages.append(tool_message1)

    # execute the 2nd tool and using the value of conversion rate
    elif tool_call['name'] == 'convert':
        
        # fetch the current argument 
        tool_call['args']['conversion_rate'] = conversion_rate

        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)


In [92]:
messages

[HumanMessage(content='what is the conversion factor between USD and INR and based on that convert 900 dollars to indian rupees', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_b2Fweh8ccEa3seesmngBAF7e', 'function': {'arguments': '{"base_currency": "USD", "target_currency": "INR"}', 'name': 'get_conversion_factor'}, 'type': 'function'}, {'id': 'call_mo8Qazijq8VbpOraSLhMKci7', 'function': {'arguments': '{"base_currency_value": 900}', 'name': 'convert'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 53, 'prompt_tokens': 108, 'total_tokens': 161, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-C2t85jXyiq2GrtDZgAqjr9s3dAvi3',

In [93]:
llm_with_tool.invoke(messages).content

'The conversion factor between USD and INR is approximately 87.6119. Therefore, converting 900 dollars to Indian Rupees gives you about ₹78,850.71.'